# Fetch Data

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../')))
import pdal
import json
from scripts.data_fetcher import FetchData
import geopandas as gpd
from shapely.geometry import Polygon
import pprint
pp = pprint.PrettyPrinter(indent=14)


## Explore the data 
Taking a sample laz and ept.json file to explore what is inside

In [3]:
with open("../data/ept.json",'r') as ept_file:
    ept_json = json.load(ept_file)

pp.pprint(ept_json)


{             'bounds': [-16816671, 11131304, -6851, -16801711, 11146264, 8109],
              'boundsConforming': [             -16813353,
                                                11131305,
                                                -28,
                                                -16805029,
                                                11146262,
                                                1286],
              'dataType': 'laszip',
              'hierarchyType': 'json',
              'points': 298602840,
              'schema': [             {             'name': 'X',
                                                    'offset': -16809191,
                                                    'scale': 0.01,
                                                    'size': 4,
                                                    'type': 'signed'},
                                      {             'name': 'Y',
                                                    'offset': 1

#### We can see from the json file that the ept.json file is indexed with 33711288742 points. The data is compressed using laszip which is a laz file format.

In [5]:
import numpy as np
import pylas

with pylas.open('../data/6-50-20-32.laz') as fh:
    print('Points from Header:', fh.header.point_count)
    las = fh.read()
    print(las)
    print('Points from data:', len(las.points))
    ground_pts = las.classification == 2
    bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
    print('Ground Point Return Number distribution:')
    for r,c in zip(bins,counts):
        print('    {}:{}'.format(r,c))

Points from Header: 46853
<LasData(1.2, point fmt: <PointFormat(1)>, 46853 points, 3 vlrs)>
Points from data: 46853
Ground Point Return Number distribution:
    1:20731
    2:779
    3:25
    4:2


## Prepare the input

In [9]:
def prepare_input(polygon_boundaries):
    MINX, MINY, MAXX, MAXY = polygon_boundaries
    polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

    grid = gpd.GeoDataFrame([polygon], columns=["geometry"])
    grid.set_crs(epsg=4326, inplace=True)
    return str(grid.loc[0,"geometry"])

polygon_bound = prepare_input([-93.756155, 41.918015, -93.747334, 41.921429])

In [10]:
polygon_bound

'POLYGON ((-93.75615500000001 41.918015, -93.75615500000001 41.921429, -93.747334 41.921429, -93.747334 41.918015, -93.75615500000001 41.918015))'

In [12]:
fd = FetchData(public_data_path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/",
region = "IA_FullState/",polygon_bound = polygon_bound)

